# RegTopK vs TopK for Fine Tuning Models on ImageNette
In this notebook, we fine tune various models on ImageNette using both TopK and RegTopK algorithms and compare the results together.

In [1]:
# Install fastai and torchvision if not already installed
# !pip install -q fastai torchvision

# Import Libraries
from fastai.vision.all import *
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, Subset
import torch
import random

# Download ImageNette (160px version, ~160MB)
path = untar_data(URLs.IMAGENETTE_160)
print(f"Data downloaded to: {path}")

# Show class folders
print("Classes:", (path/'train').ls())


Data downloaded to: /Users/alibereyhi/.fastai/data/imagenette2-160
Classes: [Path('/Users/alibereyhi/.fastai/data/imagenette2-160/train/n03394916'), Path('/Users/alibereyhi/.fastai/data/imagenette2-160/train/n03417042'), Path('/Users/alibereyhi/.fastai/data/imagenette2-160/train/.DS_Store'), Path('/Users/alibereyhi/.fastai/data/imagenette2-160/train/n03445777'), Path('/Users/alibereyhi/.fastai/data/imagenette2-160/train/n02102040'), Path('/Users/alibereyhi/.fastai/data/imagenette2-160/train/n03425413'), Path('/Users/alibereyhi/.fastai/data/imagenette2-160/train/n03888257'), Path('/Users/alibereyhi/.fastai/data/imagenette2-160/train/n03028079'), Path('/Users/alibereyhi/.fastai/data/imagenette2-160/train/n03000684'), Path('/Users/alibereyhi/.fastai/data/imagenette2-160/train/n01440764'), Path('/Users/alibereyhi/.fastai/data/imagenette2-160/train/n02979186')]


## Let's Setup Network
We set up the network of workers and share the data among them.

In [2]:
import torch
import random
import numpy as np
from torchvision.datasets import ImageFolder
import os
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

# Set the seed for fairness
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed for data shuffeling
set_seed()

# Set the device
device = torch.device("mps")#"cuda" if torch.cuda.is_available() else "cpu")

# Define transforms (resize to 128x128, normalize)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Load the training and validation sets
train_dataset = ImageFolder(root=os.path.join(path, 'train'), transform=transform)
val_dataset   = ImageFolder(root=os.path.join(path, 'val'), transform=transform)

print(f"Total training samples: {len(train_dataset)}")
print(f"Total validation samples: {len(val_dataset)}")
print(f"Classes: {train_dataset.classes}")

Total training samples: 9469
Total validation samples: 3925
Classes: ['n01440764', 'n02102040', 'n02979186', 'n03000684', 'n03028079', 'n03394916', 'n03417042', 'n03425413', 'n03445777', 'n03888257']


### Model preprocessing
Here, we prepare the model for finetuning on ImageNette

In [3]:
# Prepare model for finetuning on ImageNette
def model_loader(model):
  # Freeze all layers
  for param in model.parameters():
      param.requires_grad = False

  # Replace classifier with trainable layer
  num_classes = len(train_dataset.classes)
  model.classifier[1] = torch.nn.Linear(model.last_channel, num_classes)

  # Only the final layer is trainable
  for param in model.classifier[1].parameters():
      param.requires_grad = True

  model = model.to(device)
  print(f"device is {device}")
  return model

### Build data loaders
We build data loaders for training and validation

In [4]:
# We set the number of users
num_users = 20

# Shuffle and split indices uniformly
train_indices = list(range(len(train_dataset)))
random.shuffle(train_indices)

# Uniform splits
user_indices = torch.chunk(torch.tensor(train_indices), num_users)

# Create subset datasets and DataLoaders for each user
user_dataloaders = []

for i, indices in enumerate(user_indices):
    subset = Subset(train_dataset, indices)
    loader = DataLoader(subset, batch_size=64, shuffle=True, num_workers=2)
    user_dataloaders.append(loader)
    print(f"User {i+1}: {len(subset)} samples")

val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

User 1: 474 samples
User 2: 474 samples
User 3: 474 samples
User 4: 474 samples
User 5: 474 samples
User 6: 474 samples
User 7: 474 samples
User 8: 474 samples
User 9: 474 samples
User 10: 474 samples
User 11: 474 samples
User 12: 474 samples
User 13: 474 samples
User 14: 474 samples
User 15: 474 samples
User 16: 474 samples
User 17: 474 samples
User 18: 474 samples
User 19: 474 samples
User 20: 463 samples


## Evaluation
This is the evaluation function used to check training results.

In [5]:
def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
    return correct / total

## TopK
We now implement the training loop for TopK

In [6]:
# This is the TopK sparsifier
def topk_sparsify(tensor, k_frac=0.01):
    """Keep top-k fraction of values in tensor; set rest to zero"""
    if tensor is None:
        return None
    tensor_flat = tensor.view(-1)
    k = max(1, int(k_frac * tensor_flat.numel()))
    if k == tensor_flat.numel():
        return tensor
    topk_vals, topk_idx = torch.topk(tensor_flat.abs(), k)
    mask = torch.zeros_like(tensor_flat)
    mask[topk_idx] = 1.0
    return (tensor_flat * mask).view_as(tensor)

In [7]:
# And the training loop
def train_TopK(model, user_dataloaders, val_loader, epochs=5, lr=0.01, k_frac=0.01):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    val_accuracies = []
    val_losses = []

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0
        batch_count = 0

        for user_loader in user_dataloaders:
            for inputs, targets in user_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = F.cross_entropy(outputs, targets)
                loss.backward()

                # TopK sparsify gradients
                with torch.no_grad():
                    for param in model.parameters():
                        if param.grad is not None:
                            param.grad.data = topk_sparsify(param.grad.data, k_frac=k_frac)

                optimizer.step()
                running_loss += loss.item()
                batch_count += 1

        avg_loss = running_loss / batch_count
        val_acc = evaluate(model, val_loader)
        val_accuracies.append(val_acc)
        val_losses.append(avg_loss)

        print(f"Epoch {epoch}/{epochs} - Loss: {avg_loss:.4f} - Val Acc: {val_acc:.4f}")

    return val_accuracies, val_losses


# RegTopK
We now implement the sparsifier and training loop for RegTopK

In [8]:
# This is the RegTopK Sparsifier
class RegTopK:
    def __init__(self, model, k_frac=0.1, mu=2.0, omega=.05, Q=1e8):
        self.k_frac = k_frac
        self.mu = mu
        self.omega = omega
        self.Q = Q
        self.eps = {}         # Sparsification error (ε)
        self.prev_grad = {}   # Previous global gradient g_{t-1}
        self.a_prev = {}      # Previous accumulated gradient a_{t-1}
        self.mask = {}        # Previous mask s_{t-1}

        for name, param in model.named_parameters():
            self.eps[name] = torch.zeros_like(param.data)
            self.prev_grad[name] = torch.zeros_like(param.data)
            self.a_prev[name] = torch.zeros_like(param.data)
            self.mask[name] = torch.zeros_like(param.data)

    def sparsify(self, name, grad):
        if grad is None:
            return None

        eps = self.eps[name]
        g_prev = self.prev_grad[name]
        a_prev = self.a_prev[name]
        s_prev = self.mask[name]

        # Accumulated gradient a_t
        a_t = grad + eps

        # Posterior distortion δ
        numerator = g_prev - self.omega * a_prev
        denominator = self.omega * a_t + 1e-10  # prevent div by zero
        delta = s_prev * (numerator / denominator) + self.Q * (1 - s_prev)

        # Importance weights
        weight = torch.tanh(torch.abs(1 + delta) / self.mu)

        # Compute sparsity mask via weighted TopK
        importance = torch.abs(a_t * weight).flatten()
        k = max(1, int(self.k_frac * importance.numel()))
        topk_vals, topk_idx = torch.topk(importance, k)
        mask_flat = torch.zeros_like(importance)
        mask_flat[topk_idx] = 1.0
        mask = mask_flat.view_as(a_t)

        # Apply mask
        sparse_grad = a_t * mask

        # Update memory
        self.eps[name] = a_t - sparse_grad
        self.prev_grad[name] = grad.detach().clone()
        self.a_prev[name] = a_t.detach().clone()
        self.mask[name] = mask.detach().clone()

        return sparse_grad

In [9]:
# And this is the train loop with RegTopK
def train_regTopK(model, user_dataloaders, val_loader, epochs=5, lr=0.01, k_frac=0.01):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    omega = 1/num_users
    sparsifier = RegTopK(model, k_frac=k_frac, mu=2.0, omega=omega, Q=10000.0)

    val_accuracies = []
    val_losses = []

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0
        batch_count = 0

        for user_loader in user_dataloaders:
            for inputs, targets in user_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = F.cross_entropy(outputs, targets)
                loss.backward()

                # RegTopK sparsify gradients
                with torch.no_grad():
                  for name, param in model.named_parameters():
                    if param.grad is not None:
                      param.grad.data = sparsifier.sparsify(name, param.grad.data)

                optimizer.step()
                running_loss += loss.item()
                batch_count += 1

        avg_loss = running_loss / batch_count
        val_acc = evaluate(model, val_loader)
        val_accuracies.append(val_acc)
        val_losses.append(avg_loss)

        print(f"Epoch {epoch}/{epochs} - Loss: {avg_loss:.4f} - Val Acc: {val_acc:.4f}")

    return val_accuracies, val_losses

# MobileNetV2
Here, we finetune MobileNetV2 with both TopK and RegTopK

In [14]:
from torchvision.models import mobilenet_v2
seeds = [42, 43, 44, 45, 46, 51, 52, 53, 54, 55]
k_frac = 0.001

#### TopK

In [15]:
max_accs_topK = []
min_losses_topK = []

last_accs_topK = []
last_losses_topK = []

for seed in seeds:
    print(f"\n==== Running with seed {seed} ====")

    # Set seed
    set_seed(seed)

    # Load and prepare model
    model = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
    model = model_loader(model)

    # Train with TopK
    val_accs, val_losses = train_TopK(model, user_dataloaders, val_loader, epochs=8, lr=0.01, k_frac=k_frac)

    # Record max accuracy and min loss
    max_accs_topK.append(max(val_accs))
    min_losses_topK.append(min(val_losses))

    # Record last accuracy and loss
    last_accs_topK.append(val_accs[-1])
    last_losses_topK.append(val_losses[-1])

# Report results
print("\n===== Summary over seeds =====")
print(f"Max Val Accuracy: mean = {np.mean(max_accs_topK):.4f}, std = {np.std(max_accs_topK):.4f}")
print(f"Min Val Loss:     mean = {np.mean(min_losses_topK):.4f}, std = {np.std(min_losses_topK):.4f}")
print(f"Last-epoch Val Accuracy: mean = {np.mean(last_accs_topK):.4f}, std = {np.std(last_accs_topK):.4f}")
print(f"Last-epoch Val Loss:     mean = {np.mean(last_losses_topK):.4f}, std = {np.std(last_losses_topK):.4f}")



==== Running with seed 42 ====
device is mps
Epoch 1/8 - Loss: 1.4301 - Val Acc: 0.8494
Epoch 2/8 - Loss: 0.7041 - Val Acc: 0.8856
Epoch 3/8 - Loss: 0.5420 - Val Acc: 0.8940
Epoch 4/8 - Loss: 0.4744 - Val Acc: 0.8955
Epoch 5/8 - Loss: 0.4179 - Val Acc: 0.8986
Epoch 6/8 - Loss: 0.3905 - Val Acc: 0.9032
Epoch 7/8 - Loss: 0.3708 - Val Acc: 0.9024
Epoch 8/8 - Loss: 0.3583 - Val Acc: 0.9027

==== Running with seed 43 ====
device is mps
Epoch 1/8 - Loss: 1.4120 - Val Acc: 0.8548
Epoch 2/8 - Loss: 0.6856 - Val Acc: 0.8820
Epoch 3/8 - Loss: 0.5289 - Val Acc: 0.8917
Epoch 4/8 - Loss: 0.4536 - Val Acc: 0.8973
Epoch 5/8 - Loss: 0.4113 - Val Acc: 0.9014
Epoch 6/8 - Loss: 0.3737 - Val Acc: 0.9029
Epoch 7/8 - Loss: 0.3622 - Val Acc: 0.9060
Epoch 8/8 - Loss: 0.3481 - Val Acc: 0.9080

==== Running with seed 44 ====
device is mps
Epoch 1/8 - Loss: 1.4349 - Val Acc: 0.8520
Epoch 2/8 - Loss: 0.7034 - Val Acc: 0.8871
Epoch 3/8 - Loss: 0.5416 - Val Acc: 0.8930
Epoch 4/8 - Loss: 0.4619 - Val Acc: 0.8953
Ep

In [16]:
# Folder and file name
# save_dir = "/content/drive/MyDrive/RegTopK/MobileNetV2"
save_dir = "MobileNetV2"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"TopK_{k_frac}.txt")

# Vectors
results = {
    "Max Val Accuracy": max_accs_topK,
    "Min Val Loss": min_losses_topK,
    "Last-epoch Val Accuracy": last_accs_topK,
    "Last-epoch Val Loss": last_losses_topK,
}

# Write to file
with open(filename, "w") as f:
    f.write(f"=== TopK Results Summary (MobileNetV2) for k={k_frac} ===\n\n")
    for name, values in results.items():
        mean = np.mean(values)
        std = np.std(values)
        f.write(f"{name}:\n")
        f.write(f"  Values: {np.round(values, 4).tolist()}\n")
        f.write(f"  Mean  : {mean:.4f}\n")
        f.write(f"  Std   : {std:.4f}\n\n")

print(f"Results saved to {filename}")

Results saved to MobileNetV2/TopK_0.001.txt


#### RegTopK

In [17]:
max_accs_regTopK = []
min_losses_regTopK = []

last_accs_regTopK = []
last_losses_regTopK = []

for seed in seeds:
    print(f"\n==== Running with seed {seed} ====")

    # Set seed
    set_seed(seed)

    # Load and prepare model
    model = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
    model = model_loader(model)

    # Train with TopK
    val_accs, val_losses = train_regTopK(model, user_dataloaders, val_loader, epochs=8, lr=0.01, k_frac=k_frac)

    # Record max accuracy and min loss
    max_accs_regTopK.append(max(val_accs))
    min_losses_regTopK.append(min(val_losses))

    # Record last accuracy and loss
    last_accs_regTopK.append(val_accs[-1])
    last_losses_regTopK.append(val_losses[-1])

# Report results
print("\n===== Summary over seeds for RegTopK =====")
print(f"Max Val Accuracy: mean = {np.mean(max_accs_regTopK):.4f}, std = {np.std(max_accs_regTopK):.4f}")
print(f"Min Val Loss:     mean = {np.mean(min_losses_regTopK):.4f}, std = {np.std(min_losses_regTopK):.4f}")
print(f"Last-epoch Val Accuracy: mean = {np.mean(last_accs_regTopK):.4f}, std = {np.std(last_accs_regTopK):.4f}")
print(f"Last-epoch Val Loss:     mean = {np.mean(last_losses_regTopK):.4f}, std = {np.std(last_losses_regTopK):.4f}")



==== Running with seed 42 ====
device is mps
Epoch 1/8 - Loss: 0.7362 - Val Acc: 0.9093
Epoch 2/8 - Loss: 0.3269 - Val Acc: 0.9233
Epoch 3/8 - Loss: 0.2964 - Val Acc: 0.9254
Epoch 4/8 - Loss: 0.2876 - Val Acc: 0.9264
Epoch 5/8 - Loss: 0.2484 - Val Acc: 0.9279
Epoch 6/8 - Loss: 0.2081 - Val Acc: 0.9248
Epoch 7/8 - Loss: 0.1928 - Val Acc: 0.9287
Epoch 8/8 - Loss: 0.1795 - Val Acc: 0.9312

==== Running with seed 43 ====
device is mps
Epoch 1/8 - Loss: 0.6957 - Val Acc: 0.9070
Epoch 2/8 - Loss: 0.3312 - Val Acc: 0.9185
Epoch 3/8 - Loss: 0.3104 - Val Acc: 0.9315
Epoch 4/8 - Loss: 0.2642 - Val Acc: 0.9292
Epoch 5/8 - Loss: 0.2484 - Val Acc: 0.9310
Epoch 6/8 - Loss: 0.2134 - Val Acc: 0.9299
Epoch 7/8 - Loss: 0.2023 - Val Acc: 0.9304
Epoch 8/8 - Loss: 0.1852 - Val Acc: 0.9320

==== Running with seed 44 ====
device is mps
Epoch 1/8 - Loss: 0.7846 - Val Acc: 0.9055
Epoch 2/8 - Loss: 0.3468 - Val Acc: 0.9231
Epoch 3/8 - Loss: 0.3307 - Val Acc: 0.9287
Epoch 4/8 - Loss: 0.2872 - Val Acc: 0.9259
Ep

In [18]:
# Folder and file name
# save_dir = "/content/drive/MyDrive/RegTopK/MobileNetV2"
save_dir = "MobileNetV2"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"RegTopK_{k_frac}.txt")

# Vectors
results = {
    "Max Val Accuracy": max_accs_regTopK,
    "Min Val Loss": min_losses_regTopK,
    "Last-epoch Val Accuracy": last_accs_regTopK,
    "Last-epoch Val Loss": last_losses_regTopK,
}

# Write to file
with open(filename, "w") as f:
    f.write(f"=== RegTopK Results Summary (MobileNetV2) for k={k_frac} ===\n\n")
    for name, values in results.items():
        mean = np.mean(values)
        std = np.std(values)
        f.write(f"{name}:\n")
        f.write(f"  Values: {np.round(values, 4).tolist()}\n")
        f.write(f"  Mean  : {mean:.4f}\n")
        f.write(f"  Std   : {std:.4f}\n\n")

print(f"Results saved to {filename}")

Results saved to MobileNetV2/RegTopK_0.001.txt


#### Check Statistical Significance

In [19]:
from scipy.stats import ttest_rel, wilcoxon

# Paired t-test
t_stat, p_val_acc = ttest_rel(max_accs_regTopK, max_accs_topK)
print(f"Paired t-test on Accuracy: p = {p_val_acc:.5f}")

# Paired t-test
t_stat, p_val_loss = ttest_rel(min_losses_regTopK, min_losses_topK)
print(f"Paired t-test on loss: p = {p_val_loss:.5f}")

# Wilcoxon signed-rank test
w_stat, p_val_w_acc = wilcoxon(max_accs_regTopK, max_accs_topK)
print(f"Wilcoxon test on Accuracy: p = {p_val_w_acc:.5f}")

w_stat, p_val_w_loss = wilcoxon(min_losses_regTopK, min_losses_topK)
print(f"Wilcoxon test on loss: p = {p_val_w_loss:.5f}")

Paired t-test on Accuracy: p = 0.00000
Paired t-test on loss: p = 0.00000
Wilcoxon test on Accuracy: p = 0.00195
Wilcoxon test on loss: p = 0.00195


In [26]:
save_dir = "MobileNetV2"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"p_values_{k_frac}.txt")

with open(filename, "w") as f:
    f.write(f"=== Statistical Significance (p-values) for k={k_frac} ===\n\n")
    f.write(f"Paired t-test on Accuracy: p = {p_val_acc:.5f}\n")
    f.write(f"Paired t-test on Loss:     p = {p_val_loss:.5f}\n")
    f.write(f"Wilcoxon test on Accuracy: p = {p_val_w_acc:.5f}\n")
    f.write(f"Wilcoxon test on Loss:     p = {p_val_w_loss:.5f}\n")

print(f"\n p-values saved to {filename}")


 p-values saved to MobileNetV2/p_values_0.001.txt


# EfficientNet-B0
Here, we finetune EfficientNet-B0 with both TopK and RegTopK

In [67]:
from torchvision.models import efficientnet_b0
seeds = [42, 43, 44, 45, 46, 51, 52, 53, 54, 55]
k_frac = 0.01

In [68]:
# Prepare model for finetuning on ImageNette
def model_loader(model):
  # Freeze all layers
  for param in model.parameters():
      param.requires_grad = False

  # Replace classifier with trainable layer
  num_classes = len(train_dataset.classes)
  last_channel = model.classifier[1].in_features

  model.classifier[1] = torch.nn.Linear(last_channel, num_classes)

  # Only the final layer is trainable
  for param in model.classifier[1].parameters():
      param.requires_grad = True

  model = model.to(device)
  print(f"device is {device}")
  return model

#### TopK

In [69]:
max_accs_topK = []
min_losses_topK = []

last_accs_topK = []
last_losses_topK = []

for seed in seeds:
    print(f"\n==== Running with seed {seed} ====")

    # Set seed
    set_seed(seed)

    # Load and prepare model
    model = efficientnet_b0(weights=EfficientNet_B0_Weights)
    model = model_loader(model)

    # Train with TopK
    val_accs, val_losses = train_TopK(model, user_dataloaders, val_loader, epochs=8, lr=0.01, k_frac=k_frac)

    # Record max accuracy and min loss
    max_accs_topK.append(max(val_accs))
    min_losses_topK.append(min(val_losses))

    # Record last accuracy and loss
    last_accs_topK.append(val_accs[-1])
    last_losses_topK.append(val_losses[-1])

# Report results
print("\n===== Summary over seeds =====")
print(f"Max Val Accuracy: mean = {np.mean(max_accs_topK):.4f}, std = {np.std(max_accs_topK):.4f}")
print(f"Min Val Loss:     mean = {np.mean(min_losses_topK):.4f}, std = {np.std(min_losses_topK):.4f}")
print(f"Last-epoch Val Accuracy: mean = {np.mean(last_accs_topK):.4f}, std = {np.std(last_accs_topK):.4f}")
print(f"Last-epoch Val Loss:     mean = {np.mean(last_losses_topK):.4f}, std = {np.std(last_losses_topK):.4f}")


==== Running with seed 42 ====
device is mps


/Users/alibereyhi/.venvs/py311/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/8 - Loss: 1.3258 - Val Acc: 0.8454
Epoch 2/8 - Loss: 0.6927 - Val Acc: 0.8688
Epoch 3/8 - Loss: 0.5707 - Val Acc: 0.8752
Epoch 4/8 - Loss: 0.5118 - Val Acc: 0.8797
Epoch 5/8 - Loss: 0.4692 - Val Acc: 0.8797
Epoch 6/8 - Loss: 0.4389 - Val Acc: 0.8820
Epoch 7/8 - Loss: 0.4323 - Val Acc: 0.8836
Epoch 8/8 - Loss: 0.4222 - Val Acc: 0.8808

==== Running with seed 43 ====
device is mps
Epoch 1/8 - Loss: 1.3135 - Val Acc: 0.8527
Epoch 2/8 - Loss: 0.6864 - Val Acc: 0.8711
Epoch 3/8 - Loss: 0.5658 - Val Acc: 0.8716
Epoch 4/8 - Loss: 0.5078 - Val Acc: 0.8782
Epoch 5/8 - Loss: 0.4785 - Val Acc: 0.8797
Epoch 6/8 - Loss: 0.4491 - Val Acc: 0.8777
Epoch 7/8 - Loss: 0.4270 - Val Acc: 0.8780
Epoch 8/8 - Loss: 0.4154 - Val Acc: 0.8782

==== Running with seed 44 ====
device is mps
Epoch 1/8 - Loss: 1.3434 - Val Acc: 0.8510
Epoch 2/8 - Loss: 0.6963 - Val Acc: 0.8701
Epoch 3/8 - Loss: 0.5753 - Val Acc: 0.8757
Epoch 4/8 - Loss: 0.5014 - Val Acc: 0.8797
Epoch 5/8 - Loss: 0.4689 - Val Acc: 0.8790
Epoch

In [70]:
# Folder and file name
# save_dir = "/content/drive/MyDrive/RegTopK/MobileNetV2"
save_dir = "EfficientNet-B0"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"TopK_{k_frac}.txt")

# Vectors
results = {
    "Max Val Accuracy": max_accs_topK,
    "Min Val Loss": min_losses_topK,
    "Last-epoch Val Accuracy": last_accs_topK,
    "Last-epoch Val Loss": last_losses_topK,
}

# Write to file
with open(filename, "w") as f:
    f.write(f"=== TopK Results Summary (EfficientNet-B0) for k={k_frac} ===\n\n")
    for name, values in results.items():
        mean = np.mean(values)
        std = np.std(values)
        f.write(f"{name}:\n")
        f.write(f"  Values: {np.round(values, 4).tolist()}\n")
        f.write(f"  Mean  : {mean:.4f}\n")
        f.write(f"  Std   : {std:.4f}\n\n")

print(f"Results saved to {filename}")

Results saved to EfficientNet-B0/TopK_0.01.txt


#### RegTopK

In [71]:
max_accs_regTopK = []
min_losses_regTopK = []

last_accs_regTopK = []
last_losses_regTopK = []

for seed in seeds:
    print(f"\n==== Running with seed {seed} ====")

    # Set seed
    set_seed(seed)

    # Load and prepare model
    model = efficientnet_b0(weights=EfficientNet_B0_Weights)
    model = model_loader(model)

    # Train with TopK
    val_accs, val_losses = train_regTopK(model, user_dataloaders, val_loader, epochs=8, lr=0.01, k_frac=k_frac)

    # Record max accuracy and min loss
    max_accs_regTopK.append(max(val_accs))
    min_losses_regTopK.append(min(val_losses))

    # Record last accuracy and loss
    last_accs_regTopK.append(val_accs[-1])
    last_losses_regTopK.append(val_losses[-1])

# Report results
print("\n===== Summary over seeds for RegTopK =====")
print(f"Max Val Accuracy: mean = {np.mean(max_accs_regTopK):.4f}, std = {np.std(max_accs_regTopK):.4f}")
print(f"Min Val Loss:     mean = {np.mean(min_losses_regTopK):.4f}, std = {np.std(min_losses_regTopK):.4f}")
print(f"Last-epoch Val Accuracy: mean = {np.mean(last_accs_regTopK):.4f}, std = {np.std(last_accs_regTopK):.4f}")
print(f"Last-epoch Val Loss:     mean = {np.mean(last_losses_regTopK):.4f}, std = {np.std(last_losses_regTopK):.4f}")


==== Running with seed 42 ====
device is mps
Epoch 1/8 - Loss: 0.7892 - Val Acc: 0.8803
Epoch 2/8 - Loss: 0.3916 - Val Acc: 0.8948
Epoch 3/8 - Loss: 0.3513 - Val Acc: 0.8958
Epoch 4/8 - Loss: 0.3381 - Val Acc: 0.8978
Epoch 5/8 - Loss: 0.3050 - Val Acc: 0.8938
Epoch 6/8 - Loss: 0.2880 - Val Acc: 0.8991
Epoch 7/8 - Loss: 0.2924 - Val Acc: 0.8986
Epoch 8/8 - Loss: 0.2895 - Val Acc: 0.8940

==== Running with seed 43 ====
device is mps
Epoch 1/8 - Loss: 0.7872 - Val Acc: 0.8833
Epoch 2/8 - Loss: 0.3878 - Val Acc: 0.8955
Epoch 3/8 - Loss: 0.3475 - Val Acc: 0.8907
Epoch 4/8 - Loss: 0.3321 - Val Acc: 0.9009
Epoch 5/8 - Loss: 0.3236 - Val Acc: 0.9006
Epoch 6/8 - Loss: 0.3074 - Val Acc: 0.8989
Epoch 7/8 - Loss: 0.2823 - Val Acc: 0.8963
Epoch 8/8 - Loss: 0.2829 - Val Acc: 0.8963

==== Running with seed 44 ====
device is mps
Epoch 1/8 - Loss: 0.8071 - Val Acc: 0.8775
Epoch 2/8 - Loss: 0.3873 - Val Acc: 0.8915
Epoch 3/8 - Loss: 0.3556 - Val Acc: 0.8943
Epoch 4/8 - Loss: 0.3146 - Val Acc: 0.8963
Ep

In [72]:
# Folder and file name
save_dir = "EfficientNet-B0"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"RegTopK_{k_frac}.txt")

# Vectors
results = {
    "Max Val Accuracy": max_accs_regTopK,
    "Min Val Loss": min_losses_regTopK,
    "Last-epoch Val Accuracy": last_accs_regTopK,
    "Last-epoch Val Loss": last_losses_regTopK,
}

# Write to file
with open(filename, "w") as f:
    f.write(f"=== RegTopK Results Summary (EfficientNet-B0) for k={k_frac} ===\n\n")
    for name, values in results.items():
        mean = np.mean(values)
        std = np.std(values)
        f.write(f"{name}:\n")
        f.write(f"  Values: {np.round(values, 4).tolist()}\n")
        f.write(f"  Mean  : {mean:.4f}\n")
        f.write(f"  Std   : {std:.4f}\n\n")

print(f"Results saved to {filename}")

Results saved to EfficientNet-B0/RegTopK_0.01.txt


#### Check Statistical Significance 
We next compute p_values

In [73]:
from scipy.stats import ttest_rel, wilcoxon

# Paired t-test
t_stat, p_val_acc = ttest_rel(max_accs_regTopK, max_accs_topK)
print(f"Paired t-test on Accuracy: p = {p_val_acc:.5f}")

# Paired t-test
t_stat, p_val_loss = ttest_rel(min_losses_regTopK, min_losses_topK)
print(f"Paired t-test on loss: p = {p_val_loss:.5f}")

# Wilcoxon signed-rank test
w_stat, p_val_w_acc = wilcoxon(max_accs_regTopK, max_accs_topK)
print(f"Wilcoxon test on Accuracy: p = {p_val_w_acc:.5f}")

w_stat, p_val_w_loss = wilcoxon(min_losses_regTopK, min_losses_topK)
print(f"Wilcoxon test on loss: p = {p_val_w_loss:.5f}")

Paired t-test on Accuracy: p = 0.00000
Paired t-test on loss: p = 0.00000
Wilcoxon test on Accuracy: p = 0.00195
Wilcoxon test on loss: p = 0.00195


In [74]:
save_dir = "EfficientNet-B0"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"p_values_{k_frac}.txt")

with open(filename, "w") as f:
    f.write(f"=== Statistical Significance (p-values) for k={k_frac} ===\n\n")
    f.write(f"Paired t-test on Accuracy: p = {p_val_acc:.5f}\n")
    f.write(f"Paired t-test on Loss:     p = {p_val_loss:.5f}\n")
    f.write(f"Wilcoxon test on Accuracy: p = {p_val_w_acc:.5f}\n")
    f.write(f"Wilcoxon test on Loss:     p = {p_val_w_loss:.5f}\n")

print(f"\n p-values saved to {filename}")


 p-values saved to EfficientNet-B0/p_values_0.01.txt


# ShuffleNetV2
Here, we finetune ShuffleNetV2 with both TopK and RegTopK

In [78]:
from torchvision.models import shufflenet_v2_x1_0

seeds = [42, 43, 44, 45, 46, 51, 52, 53, 54, 55]
k_frac = 0.01

In [79]:
# Prepare model for finetuning on ImageNette
model = shufflenet_v2_x1_0(weights=ShuffleNet_V2_X1_0_Weights)

def model_loader(model):
  # Freeze all layers
  for param in model.parameters():
      param.requires_grad = False

  num_classes = len(train_dataset.classes)

  # Replace the final classification layer
  in_features = model.fc.in_features
  model.fc = torch.nn.Linear(in_features, num_classes)

  # Only the final layer is trainable
  for param in model.fc.parameters():
      param.requires_grad = True

  model = model.to(device)
  print(f"device is {device}")
  return model

#### TopK

In [80]:
max_accs_topK = []
min_losses_topK = []

last_accs_topK = []
last_losses_topK = []

for seed in seeds:
    print(f"\n==== Running with seed {seed} ====")

    # Set seed
    set_seed(seed)

    # Load and prepare model
    model = shufflenet_v2_x1_0(weights=ShuffleNet_V2_X1_0_Weights)
    model = model_loader(model)

    # Train with TopK
    val_accs, val_losses = train_TopK(model, user_dataloaders, val_loader, epochs=8, lr=0.01, k_frac=k_frac)

    # Record max accuracy and min loss
    max_accs_topK.append(max(val_accs))
    min_losses_topK.append(min(val_losses))

    # Record last accuracy and loss
    last_accs_topK.append(val_accs[-1])
    last_losses_topK.append(val_losses[-1])

# Report results
print("\n===== Summary over seeds =====")
print(f"Max Val Accuracy: mean = {np.mean(max_accs_topK):.4f}, std = {np.std(max_accs_topK):.4f}")
print(f"Min Val Loss:     mean = {np.mean(min_losses_topK):.4f}, std = {np.std(min_losses_topK):.4f}")
print(f"Last-epoch Val Accuracy: mean = {np.mean(last_accs_topK):.4f}, std = {np.std(last_accs_topK):.4f}")
print(f"Last-epoch Val Loss:     mean = {np.mean(last_losses_topK):.4f}, std = {np.std(last_losses_topK):.4f}")


==== Running with seed 42 ====
device is mps
Epoch 1/8 - Loss: 2.2918 - Val Acc: 0.3954
Epoch 2/8 - Loss: 2.2613 - Val Acc: 0.5159
Epoch 3/8 - Loss: 2.2320 - Val Acc: 0.6117
Epoch 4/8 - Loss: 2.2024 - Val Acc: 0.6978
Epoch 5/8 - Loss: 2.1751 - Val Acc: 0.7152
Epoch 6/8 - Loss: 2.1467 - Val Acc: 0.7294
Epoch 7/8 - Loss: 2.1191 - Val Acc: 0.7396
Epoch 8/8 - Loss: 2.0923 - Val Acc: 0.7518

==== Running with seed 43 ====
device is mps
Epoch 1/8 - Loss: 2.2890 - Val Acc: 0.3083
Epoch 2/8 - Loss: 2.2590 - Val Acc: 0.5289
Epoch 3/8 - Loss: 2.2291 - Val Acc: 0.6227
Epoch 4/8 - Loss: 2.2003 - Val Acc: 0.6476
Epoch 5/8 - Loss: 2.1713 - Val Acc: 0.6932
Epoch 6/8 - Loss: 2.1431 - Val Acc: 0.7292
Epoch 7/8 - Loss: 2.1165 - Val Acc: 0.7582
Epoch 8/8 - Loss: 2.0892 - Val Acc: 0.7503

==== Running with seed 44 ====
device is mps
Epoch 1/8 - Loss: 2.2903 - Val Acc: 0.3786
Epoch 2/8 - Loss: 2.2591 - Val Acc: 0.5664
Epoch 3/8 - Loss: 2.2302 - Val Acc: 0.6813
Epoch 4/8 - Loss: 2.2006 - Val Acc: 0.7284
Ep

In [81]:
# Folder and file name
# save_dir = "/content/drive/MyDrive/RegTopK/MobileNetV2"
save_dir = "ShuffleNetV2"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"TopK_{k_frac}.txt")

# Vectors
results = {
    "Max Val Accuracy": max_accs_topK,
    "Min Val Loss": min_losses_topK,
    "Last-epoch Val Accuracy": last_accs_topK,
    "Last-epoch Val Loss": last_losses_topK,
}

# Write to file
with open(filename, "w") as f:
    f.write(f"=== TopK Results Summary (ShuffleNetV2) for k={k_frac} ===\n\n")
    for name, values in results.items():
        mean = np.mean(values)
        std = np.std(values)
        f.write(f"{name}:\n")
        f.write(f"  Values: {np.round(values, 4).tolist()}\n")
        f.write(f"  Mean  : {mean:.4f}\n")
        f.write(f"  Std   : {std:.4f}\n\n")

print(f"Results saved to {filename}")

Results saved to ShuffleNetV2/TopK_0.01.txt


#### RegTopK

In [82]:
max_accs_regTopK = []
min_losses_regTopK = []

last_accs_regTopK = []
last_losses_regTopK = []

for seed in seeds:
    print(f"\n==== Running with seed {seed} ====")

    # Set seed
    set_seed(seed)

    # Load and prepare model
    model = shufflenet_v2_x1_0(weights=ShuffleNet_V2_X1_0_Weights)
    model = model_loader(model)

    # Train with TopK
    val_accs, val_losses = train_regTopK(model, user_dataloaders, val_loader, epochs=8, lr=0.01, k_frac=k_frac)

    # Record max accuracy and min loss
    max_accs_regTopK.append(max(val_accs))
    min_losses_regTopK.append(min(val_losses))

    # Record last accuracy and loss
    last_accs_regTopK.append(val_accs[-1])
    last_losses_regTopK.append(val_losses[-1])

# Report results
print("\n===== Summary over seeds for RegTopK =====")
print(f"Max Val Accuracy: mean = {np.mean(max_accs_regTopK):.4f}, std = {np.std(max_accs_regTopK):.4f}")
print(f"Min Val Loss:     mean = {np.mean(min_losses_regTopK):.4f}, std = {np.std(min_losses_regTopK):.4f}")
print(f"Last-epoch Val Accuracy: mean = {np.mean(last_accs_regTopK):.4f}, std = {np.std(last_accs_regTopK):.4f}")
print(f"Last-epoch Val Loss:     mean = {np.mean(last_losses_regTopK):.4f}, std = {np.std(last_losses_regTopK):.4f}")


==== Running with seed 42 ====
device is mps
Epoch 1/8 - Loss: 2.2565 - Val Acc: 0.6359
Epoch 2/8 - Loss: 2.1213 - Val Acc: 0.7590
Epoch 3/8 - Loss: 1.9920 - Val Acc: 0.7934
Epoch 4/8 - Loss: 1.8716 - Val Acc: 0.8087
Epoch 5/8 - Loss: 1.7682 - Val Acc: 0.8183
Epoch 6/8 - Loss: 1.6708 - Val Acc: 0.8224
Epoch 7/8 - Loss: 1.5821 - Val Acc: 0.8194
Epoch 8/8 - Loss: 1.5040 - Val Acc: 0.8280

==== Running with seed 43 ====
device is mps
Epoch 1/8 - Loss: 2.2540 - Val Acc: 0.6199
Epoch 2/8 - Loss: 2.1201 - Val Acc: 0.7603
Epoch 3/8 - Loss: 1.9914 - Val Acc: 0.8043
Epoch 4/8 - Loss: 1.8741 - Val Acc: 0.8056
Epoch 5/8 - Loss: 1.7657 - Val Acc: 0.8143
Epoch 6/8 - Loss: 1.6697 - Val Acc: 0.8145
Epoch 7/8 - Loss: 1.5849 - Val Acc: 0.8224
Epoch 8/8 - Loss: 1.5064 - Val Acc: 0.8275

==== Running with seed 44 ====
device is mps
Epoch 1/8 - Loss: 2.2552 - Val Acc: 0.6145
Epoch 2/8 - Loss: 2.1201 - Val Acc: 0.7526
Epoch 3/8 - Loss: 1.9925 - Val Acc: 0.7980
Epoch 4/8 - Loss: 1.8731 - Val Acc: 0.8084
Ep

In [83]:
# Folder and file name
save_dir = "ShuffleNetV2"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"RegTopK_{k_frac}.txt")

# Vectors
results = {
    "Max Val Accuracy": max_accs_regTopK,
    "Min Val Loss": min_losses_regTopK,
    "Last-epoch Val Accuracy": last_accs_regTopK,
    "Last-epoch Val Loss": last_losses_regTopK,
}

# Write to file
with open(filename, "w") as f:
    f.write(f"=== RegTopK Results Summary (ShuffleNetV2) for k={k_frac} ===\n\n")
    for name, values in results.items():
        mean = np.mean(values)
        std = np.std(values)
        f.write(f"{name}:\n")
        f.write(f"  Values: {np.round(values, 4).tolist()}\n")
        f.write(f"  Mean  : {mean:.4f}\n")
        f.write(f"  Std   : {std:.4f}\n\n")

print(f"Results saved to {filename}")

Results saved to ShuffleNetV2/RegTopK_0.01.txt


#### Check Statistical Significance
We now compute p_values

In [84]:
from scipy.stats import ttest_rel, wilcoxon

# Paired t-test
t_stat, p_val_acc = ttest_rel(max_accs_regTopK, max_accs_topK)
print(f"Paired t-test on Accuracy: p = {p_val_acc:.5f}")

# Paired t-test
t_stat, p_val_loss = ttest_rel(min_losses_regTopK, min_losses_topK)
print(f"Paired t-test on loss: p = {p_val_loss:.5f}")

# Wilcoxon signed-rank test
w_stat, p_val_w_acc = wilcoxon(max_accs_regTopK, max_accs_topK)
print(f"Wilcoxon test on Accuracy: p = {p_val_w_acc:.5f}")

w_stat, p_val_w_loss = wilcoxon(min_losses_regTopK, min_losses_topK)
print(f"Wilcoxon test on loss: p = {p_val_w_loss:.5f}")

Paired t-test on Accuracy: p = 0.00000
Paired t-test on loss: p = 0.00000
Wilcoxon test on Accuracy: p = 0.00195
Wilcoxon test on loss: p = 0.00195


In [85]:
save_dir = "ShuffleNetV2"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"p_values_{k_frac}.txt")

with open(filename, "w") as f:
    f.write(f"=== Statistical Significance (p-values) for k={k_frac} ===\n\n")
    f.write(f"Paired t-test on Accuracy: p = {p_val_acc:.5f}\n")
    f.write(f"Paired t-test on Loss:     p = {p_val_loss:.5f}\n")
    f.write(f"Wilcoxon test on Accuracy: p = {p_val_w_acc:.5f}\n")
    f.write(f"Wilcoxon test on Loss:     p = {p_val_w_loss:.5f}\n")

print(f"\n p-values saved to {filename}")


 p-values saved to ShuffleNetV2/p_values_0.01.txt


# SqueezeNet 1.0

In [89]:
from torchvision.models import squeezenet1_0

seeds = [42, 43, 44, 45, 46, 51, 52, 53, 54, 55]
k_frac = 0.01

In [90]:
# Prepare model for finetuning on ImageNette
model = squeezenet1_0(weights=SqueezeNet1_0_Weights)

def model_loader(model):
  # Freeze all layers
  for param in model.parameters():
      param.requires_grad = False

  num_classes = len(train_dataset.classes)

  # Replace the final conv layer to match num_classes
  model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))


  # Only the final layer is trainable
  for param in model.classifier[1].parameters():
      param.requires_grad = True

  model = model.to(device)
  print(f"device is {device}")
  return model

#### TopK

In [91]:
max_accs_topK = []
min_losses_topK = []

last_accs_topK = []
last_losses_topK = []

for seed in seeds:
    print(f"\n==== Running with seed {seed} ====")

    # Set seed
    set_seed(seed)

    # Load and prepare model
    model = squeezenet1_0(weights=SqueezeNet1_0_Weights)
    model = model_loader(model)

    # Train with TopK
    val_accs, val_losses = train_TopK(model, user_dataloaders, val_loader, epochs=8, lr=0.01, k_frac=k_frac)

    # Record max accuracy and min loss
    max_accs_topK.append(max(val_accs))
    min_losses_topK.append(min(val_losses))

    # Record last accuracy and loss
    last_accs_topK.append(val_accs[-1])
    last_losses_topK.append(val_losses[-1])

# Report results
print("\n===== Summary over seeds =====")
print(f"Max Val Accuracy: mean = {np.mean(max_accs_topK):.4f}, std = {np.std(max_accs_topK):.4f}")
print(f"Min Val Loss:     mean = {np.mean(min_losses_topK):.4f}, std = {np.std(min_losses_topK):.4f}")
print(f"Last-epoch Val Accuracy: mean = {np.mean(last_accs_topK):.4f}, std = {np.std(last_accs_topK):.4f}")
print(f"Last-epoch Val Loss:     mean = {np.mean(last_losses_topK):.4f}, std = {np.std(last_losses_topK):.4f}")


==== Running with seed 42 ====
device is mps
Epoch 1/8 - Loss: 1.0235 - Val Acc: 0.7776
Epoch 2/8 - Loss: 0.7155 - Val Acc: 0.7980
Epoch 3/8 - Loss: 0.6434 - Val Acc: 0.8130
Epoch 4/8 - Loss: 0.6162 - Val Acc: 0.8222
Epoch 5/8 - Loss: 0.5999 - Val Acc: 0.8313
Epoch 6/8 - Loss: 0.5784 - Val Acc: 0.8280
Epoch 7/8 - Loss: 0.5810 - Val Acc: 0.8176
Epoch 8/8 - Loss: 0.5677 - Val Acc: 0.8293

==== Running with seed 43 ====
device is mps
Epoch 1/8 - Loss: 1.0610 - Val Acc: 0.7771
Epoch 2/8 - Loss: 0.7150 - Val Acc: 0.8043
Epoch 3/8 - Loss: 0.6746 - Val Acc: 0.8056
Epoch 4/8 - Loss: 0.6510 - Val Acc: 0.8122
Epoch 5/8 - Loss: 0.6008 - Val Acc: 0.8178
Epoch 6/8 - Loss: 0.5991 - Val Acc: 0.7952
Epoch 7/8 - Loss: 0.5845 - Val Acc: 0.8102
Epoch 8/8 - Loss: 0.5674 - Val Acc: 0.8331

==== Running with seed 44 ====
device is mps
Epoch 1/8 - Loss: 1.1386 - Val Acc: 0.7753
Epoch 2/8 - Loss: 0.7381 - Val Acc: 0.8025
Epoch 3/8 - Loss: 0.6602 - Val Acc: 0.8158
Epoch 4/8 - Loss: 0.6334 - Val Acc: 0.8211
Ep

In [92]:
# Folder and file name
save_dir = "SqueezeNet1.0"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"TopK_{k_frac}.txt")

# Vectors
results = {
    "Max Val Accuracy": max_accs_topK,
    "Min Val Loss": min_losses_topK,
    "Last-epoch Val Accuracy": last_accs_topK,
    "Last-epoch Val Loss": last_losses_topK,
}

# Write to file
with open(filename, "w") as f:
    f.write(f"=== TopK Results Summary (SqueezeNet1.0) for k={k_frac} ===\n\n")
    for name, values in results.items():
        mean = np.mean(values)
        std = np.std(values)
        f.write(f"{name}:\n")
        f.write(f"  Values: {np.round(values, 4).tolist()}\n")
        f.write(f"  Mean  : {mean:.4f}\n")
        f.write(f"  Std   : {std:.4f}\n\n")

print(f"Results saved to {filename}")

Results saved to SqueezeNet1.0/TopK_0.01.txt


#### RegTopK

In [93]:
max_accs_regTopK = []
min_losses_regTopK = []

last_accs_regTopK = []
last_losses_regTopK = []

for seed in seeds:
    print(f"\n==== Running with seed {seed} ====")

    # Set seed
    set_seed(seed)

    # Load and prepare model
    model = squeezenet1_0(weights=SqueezeNet1_0_Weights)
    model = model_loader(model)

    # Train with TopK
    val_accs, val_losses = train_regTopK(model, user_dataloaders, val_loader, epochs=8, lr=0.01, k_frac=k_frac)

    # Record max accuracy and min loss
    max_accs_regTopK.append(max(val_accs))
    min_losses_regTopK.append(min(val_losses))

    # Record last accuracy and loss
    last_accs_regTopK.append(val_accs[-1])
    last_losses_regTopK.append(val_losses[-1])

# Report results
print("\n===== Summary over seeds for RegTopK =====")
print(f"Max Val Accuracy: mean = {np.mean(max_accs_regTopK):.4f}, std = {np.std(max_accs_regTopK):.4f}")
print(f"Min Val Loss:     mean = {np.mean(min_losses_regTopK):.4f}, std = {np.std(min_losses_regTopK):.4f}")
print(f"Last-epoch Val Accuracy: mean = {np.mean(last_accs_regTopK):.4f}, std = {np.std(last_accs_regTopK):.4f}")
print(f"Last-epoch Val Loss:     mean = {np.mean(last_losses_regTopK):.4f}, std = {np.std(last_losses_regTopK):.4f}")


==== Running with seed 42 ====
device is mps
Epoch 1/8 - Loss: 0.8963 - Val Acc: 0.8275
Epoch 2/8 - Loss: 0.5544 - Val Acc: 0.8487
Epoch 3/8 - Loss: 0.4661 - Val Acc: 0.8637
Epoch 4/8 - Loss: 0.4333 - Val Acc: 0.8690
Epoch 5/8 - Loss: 0.4351 - Val Acc: 0.8665
Epoch 6/8 - Loss: 0.4246 - Val Acc: 0.8517
Epoch 7/8 - Loss: 0.4123 - Val Acc: 0.8502
Epoch 8/8 - Loss: 0.3796 - Val Acc: 0.8800

==== Running with seed 43 ====
device is mps
Epoch 1/8 - Loss: 0.9970 - Val Acc: 0.8288
Epoch 2/8 - Loss: 0.5567 - Val Acc: 0.8324
Epoch 3/8 - Loss: 0.4917 - Val Acc: 0.8573
Epoch 4/8 - Loss: 0.4548 - Val Acc: 0.8507
Epoch 5/8 - Loss: 0.4117 - Val Acc: 0.8624
Epoch 6/8 - Loss: 0.4176 - Val Acc: 0.8642
Epoch 7/8 - Loss: 0.4034 - Val Acc: 0.8581
Epoch 8/8 - Loss: 0.3914 - Val Acc: 0.8609

==== Running with seed 44 ====
device is mps
Epoch 1/8 - Loss: 1.0053 - Val Acc: 0.8181
Epoch 2/8 - Loss: 0.5816 - Val Acc: 0.8464
Epoch 3/8 - Loss: 0.4902 - Val Acc: 0.8591
Epoch 4/8 - Loss: 0.4559 - Val Acc: 0.8683
Ep

In [94]:
# Folder and file name
save_dir = "SqueezeNet1.0"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"RegTopK_{k_frac}.txt")

# Vectors
results = {
    "Max Val Accuracy": max_accs_regTopK,
    "Min Val Loss": min_losses_regTopK,
    "Last-epoch Val Accuracy": last_accs_regTopK,
    "Last-epoch Val Loss": last_losses_regTopK,
}

# Write to file
with open(filename, "w") as f:
    f.write(f"=== RegTopK Results Summary (SqueezeNet1.0) for k={k_frac} ===\n\n")
    for name, values in results.items():
        mean = np.mean(values)
        std = np.std(values)
        f.write(f"{name}:\n")
        f.write(f"  Values: {np.round(values, 4).tolist()}\n")
        f.write(f"  Mean  : {mean:.4f}\n")
        f.write(f"  Std   : {std:.4f}\n\n")

print(f"Results saved to {filename}")

Results saved to SqueezeNet1.0/RegTopK_0.01.txt


#### Checking Statistical Significance

In [95]:
from scipy.stats import ttest_rel, wilcoxon

# Paired t-test
t_stat, p_val_acc = ttest_rel(max_accs_regTopK, max_accs_topK)
print(f"Paired t-test on Accuracy: p = {p_val_acc:.5f}")

# Paired t-test
t_stat, p_val_loss = ttest_rel(min_losses_regTopK, min_losses_topK)
print(f"Paired t-test on loss: p = {p_val_loss:.5f}")

# Wilcoxon signed-rank test
w_stat, p_val_w_acc = wilcoxon(max_accs_regTopK, max_accs_topK)
print(f"Wilcoxon test on Accuracy: p = {p_val_w_acc:.5f}")

w_stat, p_val_w_loss = wilcoxon(min_losses_regTopK, min_losses_topK)
print(f"Wilcoxon test on loss: p = {p_val_w_loss:.5f}")

Paired t-test on Accuracy: p = 0.00000
Paired t-test on loss: p = 0.00000
Wilcoxon test on Accuracy: p = 0.00195
Wilcoxon test on loss: p = 0.00195


In [96]:
save_dir = "SqueezeNet1.0"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"p_values_{k_frac}.txt")

with open(filename, "w") as f:
    f.write(f"=== Statistical Significance (p-values) for k={k_frac} ===\n\n")
    f.write(f"Paired t-test on Accuracy: p = {p_val_acc:.5f}\n")
    f.write(f"Paired t-test on Loss:     p = {p_val_loss:.5f}\n")
    f.write(f"Wilcoxon test on Accuracy: p = {p_val_w_acc:.5f}\n")
    f.write(f"Wilcoxon test on Loss:     p = {p_val_w_loss:.5f}\n")

print(f"\n p-values saved to {filename}")


 p-values saved to SqueezeNet1.0/p_values_0.01.txt


# ResNet-152

In [97]:
from torchvision.models import resnet152

seeds = [42, 43, 44, 45, 46, 51, 52, 53, 54, 55]
k_frac = 0.01

In [98]:
# Prepare model for finetuning on ImageNette
model = resnet152(weights=ResNet152_Weights)

def model_loader(model):
  # Freeze all layers
  for param in model.parameters():
      param.requires_grad = False

  num_classes = len(train_dataset.classes)

  # Replace the final classification layer
  in_features = model.fc.in_features
  model.fc = torch.nn.Linear(in_features, num_classes)

  # Only the final layer is trainable
  for param in model.fc.parameters():
      param.requires_grad = True

  model = model.to(device)
  print(f"device is {device}")
  return model

#### TopK

In [99]:
max_accs_topK = []
min_losses_topK = []

last_accs_topK = []
last_losses_topK = []

for seed in seeds:
    print(f"\n==== Running with seed {seed} ====")

    # Set seed
    set_seed(seed)

    # Load and prepare model
    model = resnet152(weights=ResNet152_Weights)
    model = model_loader(model)

    # Train with TopK
    val_accs, val_losses = train_TopK(model, user_dataloaders, val_loader, epochs=8, lr=0.01, k_frac=k_frac)

    # Record max accuracy and min loss
    max_accs_topK.append(max(val_accs))
    min_losses_topK.append(min(val_losses))

    # Record last accuracy and loss
    last_accs_topK.append(val_accs[-1])
    last_losses_topK.append(val_losses[-1])

# Report results
print("\n===== Summary over seeds =====")
print(f"Max Val Accuracy: mean = {np.mean(max_accs_topK):.4f}, std = {np.std(max_accs_topK):.4f}")
print(f"Min Val Loss:     mean = {np.mean(min_losses_topK):.4f}, std = {np.std(min_losses_topK):.4f}")
print(f"Last-epoch Val Accuracy: mean = {np.mean(last_accs_topK):.4f}, std = {np.std(last_accs_topK):.4f}")
print(f"Last-epoch Val Loss:     mean = {np.mean(last_losses_topK):.4f}, std = {np.std(last_losses_topK):.4f}")


==== Running with seed 42 ====
device is mps
Epoch 1/8 - Loss: 0.4621 - Val Acc: 0.9325
Epoch 2/8 - Loss: 0.2172 - Val Acc: 0.9330
Epoch 3/8 - Loss: 0.1971 - Val Acc: 0.9399
Epoch 4/8 - Loss: 0.1765 - Val Acc: 0.9409
Epoch 5/8 - Loss: 0.1697 - Val Acc: 0.9389
Epoch 6/8 - Loss: 0.1591 - Val Acc: 0.9457
Epoch 7/8 - Loss: 0.1505 - Val Acc: 0.9442
Epoch 8/8 - Loss: 0.1465 - Val Acc: 0.9391

==== Running with seed 43 ====
device is mps
Epoch 1/8 - Loss: 0.4572 - Val Acc: 0.9332
Epoch 2/8 - Loss: 0.2241 - Val Acc: 0.9401
Epoch 3/8 - Loss: 0.1875 - Val Acc: 0.9391
Epoch 4/8 - Loss: 0.1746 - Val Acc: 0.9422
Epoch 5/8 - Loss: 0.1631 - Val Acc: 0.9406
Epoch 6/8 - Loss: 0.1574 - Val Acc: 0.9417
Epoch 7/8 - Loss: 0.1491 - Val Acc: 0.9376
Epoch 8/8 - Loss: 0.1419 - Val Acc: 0.9409

==== Running with seed 44 ====
device is mps
Epoch 1/8 - Loss: 0.4644 - Val Acc: 0.9338
Epoch 2/8 - Loss: 0.2317 - Val Acc: 0.9383
Epoch 3/8 - Loss: 0.2024 - Val Acc: 0.9409
Epoch 4/8 - Loss: 0.1826 - Val Acc: 0.9411
Ep

In [100]:
# Folder and file name
save_dir = "ResNet152"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"TopK_{k_frac}.txt")

# Vectors
results = {
    "Max Val Accuracy": max_accs_topK,
    "Min Val Loss": min_losses_topK,
    "Last-epoch Val Accuracy": last_accs_topK,
    "Last-epoch Val Loss": last_losses_topK,
}

# Write to file
with open(filename, "w") as f:
    f.write(f"=== TopK Results Summary (ResNet152) for k={k_frac} ===\n\n")
    for name, values in results.items():
        mean = np.mean(values)
        std = np.std(values)
        f.write(f"{name}:\n")
        f.write(f"  Values: {np.round(values, 4).tolist()}\n")
        f.write(f"  Mean  : {mean:.4f}\n")
        f.write(f"  Std   : {std:.4f}\n\n")

print(f"Results saved to {filename}")

Results saved to ResNet152/TopK_0.01.txt


#### RegTopK

In [101]:
max_accs_regTopK = []
min_losses_regTopK = []

last_accs_regTopK = []
last_losses_regTopK = []

for seed in seeds:
    print(f"\n==== Running with seed {seed} ====")

    # Set seed
    set_seed(seed)

    # Load and prepare model
    model = resnet152(weights=ResNet152_Weights)
    model = model_loader(model)

    # Train with TopK
    val_accs, val_losses = train_regTopK(model, user_dataloaders, val_loader, epochs=8, lr=0.01, k_frac=k_frac)

    # Record max accuracy and min loss
    max_accs_regTopK.append(max(val_accs))
    min_losses_regTopK.append(min(val_losses))

    # Record last accuracy and loss
    last_accs_regTopK.append(val_accs[-1])
    last_losses_regTopK.append(val_losses[-1])

# Report results
print("\n===== Summary over seeds for RegTopK =====")
print(f"Max Val Accuracy: mean = {np.mean(max_accs_regTopK):.4f}, std = {np.std(max_accs_regTopK):.4f}")
print(f"Min Val Loss:     mean = {np.mean(min_losses_regTopK):.4f}, std = {np.std(min_losses_regTopK):.4f}")
print(f"Last-epoch Val Accuracy: mean = {np.mean(last_accs_regTopK):.4f}, std = {np.std(last_accs_regTopK):.4f}")
print(f"Last-epoch Val Loss:     mean = {np.mean(last_losses_regTopK):.4f}, std = {np.std(last_losses_regTopK):.4f}")


==== Running with seed 42 ====
device is mps
Epoch 1/8 - Loss: 0.4725 - Val Acc: 0.9144
Epoch 2/8 - Loss: 0.3010 - Val Acc: 0.9279
Epoch 3/8 - Loss: 0.2114 - Val Acc: 0.9399
Epoch 4/8 - Loss: 0.1620 - Val Acc: 0.9355
Epoch 5/8 - Loss: 0.1406 - Val Acc: 0.9414
Epoch 6/8 - Loss: 0.1473 - Val Acc: 0.9361
Epoch 7/8 - Loss: 0.1294 - Val Acc: 0.9401
Epoch 8/8 - Loss: 0.1179 - Val Acc: 0.9411

==== Running with seed 43 ====
device is mps
Epoch 1/8 - Loss: 0.4347 - Val Acc: 0.9343
Epoch 2/8 - Loss: 0.2792 - Val Acc: 0.9223
Epoch 3/8 - Loss: 0.2223 - Val Acc: 0.9259
Epoch 4/8 - Loss: 0.1845 - Val Acc: 0.9343
Epoch 5/8 - Loss: 0.1443 - Val Acc: 0.9427
Epoch 6/8 - Loss: 0.1460 - Val Acc: 0.9394
Epoch 7/8 - Loss: 0.1516 - Val Acc: 0.9386
Epoch 8/8 - Loss: 0.1187 - Val Acc: 0.9368

==== Running with seed 44 ====
device is mps
Epoch 1/8 - Loss: 0.4890 - Val Acc: 0.9177
Epoch 2/8 - Loss: 0.3645 - Val Acc: 0.9386
Epoch 3/8 - Loss: 0.2846 - Val Acc: 0.9220
Epoch 4/8 - Loss: 0.1934 - Val Acc: 0.9404
Ep

In [102]:
# Folder and file name
save_dir = "ResNet152"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"RegTopK_{k_frac}.txt")

# Vectors
results = {
    "Max Val Accuracy": max_accs_regTopK,
    "Min Val Loss": min_losses_regTopK,
    "Last-epoch Val Accuracy": last_accs_regTopK,
    "Last-epoch Val Loss": last_losses_regTopK,
}

# Write to file
with open(filename, "w") as f:
    f.write(f"=== RegTopK Results Summary (ResNet152) for k={k_frac} ===\n\n")
    for name, values in results.items():
        mean = np.mean(values)
        std = np.std(values)
        f.write(f"{name}:\n")
        f.write(f"  Values: {np.round(values, 4).tolist()}\n")
        f.write(f"  Mean  : {mean:.4f}\n")
        f.write(f"  Std   : {std:.4f}\n\n")

print(f"Results saved to {filename}")

Results saved to ResNet152/RegTopK_0.01.txt


#### Checking Statistical Significance

In [ ]:
from scipy.stats import ttest_rel, wilcoxon

# Paired t-test
t_stat, p_val_acc = ttest_rel(max_accs_regTopK, max_accs_topK)
print(f"Paired t-test on Accuracy: p = {p_val_acc:.5f}")

# Paired t-test
t_stat, p_val_loss = ttest_rel(min_losses_regTopK, min_losses_topK)
print(f"Paired t-test on loss: p = {p_val_loss:.5f}")

# Wilcoxon signed-rank test
w_stat, p_val_w_acc = wilcoxon(max_accs_regTopK, max_accs_topK)
print(f"Wilcoxon test on Accuracy: p = {p_val_w_acc:.5f}")

w_stat, p_val_w_loss = wilcoxon(min_losses_regTopK, min_losses_topK)
print(f"Wilcoxon test on loss: p = {p_val_w_loss:.5f}")

Paired t-test on Accuracy: p = 0.71050
Paired t-test on loss: p = 0.00174
Wilcoxon test on Accuracy: p = 0.79102
Wilcoxon test on loss: p = 0.00586


In [66]:
save_dir = "ResNet152"
os.makedirs(save_dir, exist_ok=True)
filename = os.path.join(save_dir, f"p_values_{k_frac}.txt")

with open(filename, "w") as f:
    f.write(f"=== Statistical Significance (p-values) for k={k_frac} ===\n\n")
    f.write(f"Paired t-test on Accuracy: p = {p_val_acc:.5f}\n")
    f.write(f"Paired t-test on Loss:     p = {p_val_loss:.5f}\n")
    f.write(f"Wilcoxon test on Accuracy: p = {p_val_w_acc:.5f}\n")
    f.write(f"Wilcoxon test on Loss:     p = {p_val_w_loss:.5f}\n")

print(f"\n p-values saved to {filename}")


 p-values saved to ResNet152/p_values_0.001.txt
